In [1]:
#pip install -r requirements.txt

# Pacote

In [1]:
import pandas as pd
import chardet
import glob
from unidecode import unidecode
import os
import Levenshtein as lev
import csv 

pd.set_option('display.max_columns', None)

# Funções

In [2]:
def detect_encoding(file_pattern_or_path, num_bytes=10000):
    """
    Detecta a codificação do arquivo ou arquivos fornecidos.
    
    Parâmetros:
        file_pattern_or_path (str): Caminho ou padrão do arquivo para detecção.
        num_bytes (int, opcional): Número de bytes para ler para a detecção. Padrão é 10000.
    
    Retorna:
        dict: Dicionário com caminho do arquivo como chave e codificação detectada como valor.
    """
    
    # Encontrar arquivos que correspondem ao padrão ou caminho fornecido.
    files = glob.glob(file_pattern_or_path)
    encodings = {}

    # Iterar sobre cada arquivo encontrado.
    for file_path in files:
        # Abrir arquivo em modo binário e ler os primeiros 'num_bytes' bytes.
        with open(file_path, 'rb') as f:
            rawdata = f.read(num_bytes)
            # Detectar a codificação do fragmento lido e armazenar no dicionário.
            encodings[file_path] = chardet.detect(rawdata)["encoding"]
    
    return encodings

def is_delayed(row):
    """
    Determina se um voo está atrasado ou pontual com base na diferença de tempo entre a partida prevista e a real.

    Parâmetros:
    row (pd.Series): Uma linha de um DataFrame que contém informações de partida do voo.

    Retorna:
    str: "Atrasado" se o voo estiver atrasado por mais de 15 minutos, caso contrário "Pontual".
    """
    # Calcula a diferença de tempo entre a partida real e a prevista.
    delta = row['Partida Real'] - row['Partida Prevista']
    # Retorna "Atrasado" se a diferença for maior que 15 minutos, senão "Pontual".
    return "Atrasado" if delta > pd.Timedelta(minutes=15) else "Pontual"

def padronizar_nome_coluna(coluna):
    """
    Padroniza o nome de uma coluna removendo acentos, transformando em minúsculas e substituindo espaços por underscores.

    Parâmetros:
    coluna (str): Nome da coluna a ser padronizado.

    Retorna:
    str: Nome da coluna padronizado.
    """
    # Remover acentos do nome da coluna.
    coluna = unidecode(coluna)
    # Transformar todas as letras em minúsculas.
    coluna = coluna.lower()
    # Substituir espaços por underscores.
    coluna = coluna.replace(' ', '_')
    return coluna

def calculate_time_delta(row, start_col, end_col):
    """
    Calcula a diferença de tempo entre duas colunas de um DataFrame.

    Parâmetros:
    row (pd.Series): Uma linha do DataFrame.
    start_col (str): Nome da coluna com o tempo inicial.
    end_col (str): Nome da coluna com o tempo final.

    Retorna:
    pd.Timedelta: Diferença de tempo entre 'start_col' e 'end_col'.
    """
    # Extrai os tempos iniciais e finais das colunas especificadas.
    start_time = row[start_col]
    end_time = row[end_col]
    
    # Verifica se algum dos tempos é NaT (Not a Time) e retorna 0 se verdadeiro.
    if pd.isna(start_time) or pd.isna(end_time):
        return 0
    
    # Calcula e retorna a diferença absoluta de tempo entre os dois tempos.
    delta = abs(end_time - start_time)
    return delta

def calculate_time_int(row, start_col, end_col):
    """
    Calcula a diferença de tempo em minutos entre duas colunas de um DataFrame.

    Parâmetros:
    row (pd.Series): Uma linha do DataFrame.
    start_col (str): Nome da coluna com o tempo inicial.
    end_col (str): Nome da coluna com o tempo final.

    Retorna:
    int: Diferença de tempo em minutos entre 'start_col' e 'end_col'.
    """
    # Extrai os tempos iniciais e finais das colunas especificadas.
    start_time = row[start_col]
    end_time = row[end_col]
    
    # Verifica se algum dos tempos é NaT (Not a Time) e retorna 0 se verdadeiro.
    if pd.isna(start_time) or pd.isna(end_time):
        return 0
    
    # Calcula a diferença de tempo, converte para segundos e depois para minutos.
    delta = abs(end_time - start_time)
    return int(delta.total_seconds() / 60)

def encontrar_coluna_similar(coluna, colunas_padrao, limiar=8):
    """
    Encontra a coluna mais similar a um dado nome de coluna, baseado em uma medida de distância.

    Args:
    coluna (str): Nome da coluna a ser comparada.
    colunas_padrao (list): Lista de nomes de colunas padrão para comparação.
    limiar (int, optional): Limiar de distância para considerar uma coluna similar. Default é 8.

    Etapas:
    1. Iterar sobre cada coluna padrão.
    2. Calcular a distância de Levenshtein entre a coluna e cada coluna padrão.
    3. Se a distância for menor ou igual ao limiar, retornar a coluna padrão correspondente.
    4. Se nenhuma coluna padrão satisfizer o critério, retornar o nome da coluna original.

    Returns:
    str: O nome da coluna mais similar dentre as colunas padrão, ou o próprio nome da coluna se nenhuma for suficientemente similar.
    """
     # Itera sobre cada coluna padrão na lista fornecida.
    for padrao in colunas_padrao:
        # Calcula a distância de Levenshtein entre a coluna e a coluna padrão atual.
        # A função lev.distance() é assumida como uma função predefinida ou importada para calcular esta distância.
        if lev.distance(coluna, padrao) <= limiar:
            # Se a distância for menor ou igual ao limiar, retorna a coluna padrão como correspondente.
            return padrao
    # Se nenhuma coluna padrão corresponder dentro do limiar, retorna o nome da coluna original.
    return coluna

# Função para detectar o delimitador mais adequado
def detectar_delimitador(filename, encoding='utf-8', num_lines=10):
    """
    Detecta o delimitador mais adequado para um arquivo CSV.

    Args:
    filename (str): Nome do arquivo a ser analisado.
    encoding (str, optional): Codificação do arquivo. Default é 'utf-8'.
    num_lines (int, optional): Número de linhas a serem lidas para a detecção do delimitador. Default é 10.

    Etapas:
    1. Definir uma lista de delimitadores possíveis.
    2. Ler as primeiras linhas do arquivo, testando cada delimitador possível.
    3. Identificar o delimitador que resulta no maior número de campos (colunas) por linha.
    4. Retornar o delimitador que maximiza o número de campos.

    Returns:
    str: O delimitador mais adequado encontrado no arquivo. Se nenhum delimitador adequado for encontrado, retorna None.
    """
    # Define uma lista de delimitadores possíveis para testar no arquivo.
    delimitadores_possiveis = [',', ';', '\t', '|']
    # Inicializa variáveis para armazenar o delimitador com o máximo de campos e o número máximo de campos.
    max_delimiter = None
    max_fields = 1

    # Abre o arquivo com o nome e codificação especificados.
    with open(filename, 'r', encoding=encoding) as file:
        # Testa cada delimitador possível.
        for delimiter in delimitadores_possiveis:
            # Retorna ao início do arquivo antes de testar um novo delimitador.
            file.seek(0)
            # Cria um leitor CSV com o delimitador atual.
            reader = csv.reader(file, delimiter=delimiter)
            # Itera sobre as linhas do arquivo até o limite especificado.
            for i, row in enumerate(reader):
                # Interrompe a leitura após um determinado número de linhas.
                if i >= num_lines:
                    break
                # Atualiza o delimitador e o número máximo de campos se este delimitador resultar em mais campos.
                if len(row) > max_fields:
                    max_fields = len(row)
                    max_delimiter = delimiter

    # Retorna o delimitador que resultou no maior número de campos por linha.
    return max_delimiter

# Carregando dados

In [3]:
# --- Carregamento dos Dados dos Aeroportos ---
# Definindo o caminho para o arquivo CSV dos glossários de aeródromos.
file_path = "dados_complementares_convertidos/glossario_de_aerodromo.csv"

# Detectando a codificação do arquivo de aeródromos para assegurar a leitura correta dos dados.
file_encodings = detect_encoding(file_path)

# Obtendo a codificação detectada para o arquivo específico.
encoding = file_encodings[file_path]

# Lendo o arquivo CSV com a codificação apropriada.
# A codificação detectada é usada para lidar com possíveis caracteres especiais nos dados.
df_aeroportos = pd.read_csv(file_path, sep=',', encoding=encoding)  # A codificação pode variar (ex: 'iso-8859-1', 'cp1252'), dependendo do arquivo.


# --- Carregamento dos Dados das Companhias Aéreas ---
# Definindo o caminho para o arquivo CSV dos glossários de empresas aéreas.
file_path = "dados_complementares_convertidos/glossario_de_empresas_aereas.csv"

# Detectando a codificação do arquivo das empresas aéreas.
file_encodings = detect_encoding(file_path)

# Obtendo a codificação detectada para este arquivo específico.
encoding = file_encodings[file_path]

# Lendo o arquivo CSV com a codificação apropriada.
# Novamente, a codificação detectada é essencial para lidar com caracteres especiais.
df_cia_aerea = pd.read_csv(file_path, sep=',', encoding=encoding)  # A codificação pode variar, semelhante ao arquivo de aeródromos.

In [4]:
# Definindo o caminho para os arquivos CSV do ano de 2019.
file_path = "2018/*.csv"

# Detectando a codificação dos arquivos CSV
file_encodings = detect_encoding(file_path)

# Criando uma lista para armazenar os DataFrames de cada arquivo CSV.
df_list = []

# Iniciando a iteração sobre todos os arquivos CSV no diretório especificado.
for filename in glob.glob(file_path):
    print(f"Carregando arquivo: {filename}")
    try:
        # Obter a codificação do arquivo
        encoding = file_encodings.get(filename, 'latin1')

        # Detectar o delimitador apropriado para cada arquivo
        delimitador = detectar_delimitador(filename, encoding=encoding)

        # Lendo o arquivo CSV com a codificação apropriada
        df = pd.read_csv(filename, encoding=encoding, sep=delimitador, low_memory=False, index_col=None)
        df_list.append(df)
    except Exception as e:
        print(f"Erro ao carregar o arquivo {filename}: {e}")

# Verificando se DataFrames foram carregados na lista.
if df_list:
    # Concatenando todos os DataFrames em um único DataFrame para análise combinada.
    combined_df = pd.concat(df_list, ignore_index=True)
    print("Todos os arquivos CSV foram carregados com sucesso.")
else:
    print("Nenhum arquivo CSV foi encontrado ou carregado.")

Carregando arquivo: 2018\vra-01_2018.csv
Carregando arquivo: 2018\vra-02_2018.csv
Carregando arquivo: 2018\vra-03_2018.csv
Carregando arquivo: 2018\vra-04_2018.csv
Carregando arquivo: 2018\vra-05_2018.csv
Carregando arquivo: 2018\vra-06_2018.csv
Carregando arquivo: 2018\vra-07_2018.csv
Carregando arquivo: 2018\vra-08_2018.csv
Carregando arquivo: 2018\vra-09_2018.csv
Carregando arquivo: 2018\vra-10_2018.csv
Carregando arquivo: 2018\vra-11_2018.csv
Carregando arquivo: 2018\vra-12_2018.csv
Todos os arquivos CSV foram carregados com sucesso.


In [5]:
mapeamento_colunas = {
    'Código Autorização (DI)': 'Codigo DI',
    'cd_di': 'Codigo DI',
    'ICAO Empresa Aerea': 'ICAO Empresa Aérea',
    'sg_empresa_icao': 'ICAO Empresa Aérea',
    'Número Voo': 'Número Voo',
    'NUmero Voo': 'Número Voo',
    'nr_voo': 'Número Voo',
    'Código Tipo Linha': 'Código Tipo Linha',
    'Codigo Tipo Linha': 'Código Tipo Linha',
    'cd_tipo_linha': 'Código Tipo Linha',
    'ICAO Aeródromo Origem': 'ICAO Aeródromo Origem',
    'ICAO Aerodromo Origem': 'ICAO Aeródromo Origem',
    'sg_icao_origem': 'ICAO Aeródromo Origem',
    'ICAO Aeródromo Destino': 'ICAO Aeródromo Destino',
    'ICAO Aerodromo Destino': 'ICAO Aeródromo Destino',
    'sg_icao_destino': 'ICAO Aeródromo Destino',
    'Partida Prevista': 'Partida Prevista',
    'dt_partida_prevista': 'Partida Prevista',
    'Partida Real': 'Partida Real',
    'Código DI': 'Codigo DI',   
    'Data Partida Real': 'Partida Real',
    'Data Chegada Real': 'Chegada Real',
    'Data Partida Prevista': 'Partida Prevista',
    'Data Chegada Prevista': 'Chegada Prevista',
    'dt_partida_real': 'Partida Real',
    'Chegada Prevista': 'Chegada Prevista',
    'dt_chegada_prevista': 'Chegada Prevista',
    'Chegada Real': 'Chegada Real',
    'dt_chegada_real': 'Chegada Real',
    'Situação Voo': 'Situação Voo',
    'Situacao Voo': 'Situação Voo',
    'situacao': 'Situação Voo',
    'Código Justificativa': 'Código Justificativa',
    'Codigo Justificativa': 'Código Justificativa',
    'cd_justificativa': 'Código Justificativa'}

In [6]:
# Colunas padrão esperadas
colunas_esperadas = [
    'ICAO Empresa Aérea', 'Número Voo', 'Código Tipo Linha',
    'ICAO Aeródromo Origem', 'ICAO Aeródromo Destino', 'Partida Prevista', 'Partida Real',
    'Chegada Prevista', 'Chegada Real', 'Situação Voo', 'Código Justificativa', 'Codigo DI'
]

# Definindo o caminho para os arquivos CSV do ano de 2019.
file_path = "2018/*.csv"

# Detectando a codificação dos arquivos CSV
file_encodings = detect_encoding(file_path)

# Criando uma lista para armazenar os DataFrames de cada arquivo CSV.
df_list = []

# Processamento dos arquivos CSV
df_list = []
for filename in glob.glob(file_path):
    print(f"Carregando arquivo: {filename}")
    try:
        encoding = file_encodings.get(filename, 'latin1')
        delimitador = detectar_delimitador(filename, encoding=encoding)
        df = pd.read_csv(filename, encoding=encoding, sep=delimitador, low_memory=False, index_col=None)

        # Renomear colunas com base no mapeamento
        df.rename(columns=mapeamento_colunas, inplace=True, errors='ignore')

        # Manter apenas as colunas esperadas
        df = df[colunas_esperadas]
        df_list.append(df)
    except Exception as e:
        print(f"Erro ao carregar o arquivo {filename}: {e}")

# Concatenando todos os DataFrames em um único DataFrame para análise combinada.
if df_list:
    combined_df = pd.concat(df_list, ignore_index=True)
    print("Todos os arquivos CSV foram carregados com sucesso.")
else:
    print("Nenhum arquivo CSV foi encontrado ou carregado.")

Carregando arquivo: 2018\vra-01_2018.csv
Carregando arquivo: 2018\vra-02_2018.csv
Carregando arquivo: 2018\vra-03_2018.csv
Carregando arquivo: 2018\vra-04_2018.csv
Carregando arquivo: 2018\vra-05_2018.csv
Carregando arquivo: 2018\vra-06_2018.csv
Carregando arquivo: 2018\vra-07_2018.csv
Carregando arquivo: 2018\vra-08_2018.csv
Carregando arquivo: 2018\vra-09_2018.csv
Carregando arquivo: 2018\vra-10_2018.csv
Carregando arquivo: 2018\vra-11_2018.csv
Carregando arquivo: 2018\vra-12_2018.csv
Todos os arquivos CSV foram carregados com sucesso.


In [7]:
df = combined_df.copy()

In [8]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1034556 entries, 0 to 1034555
Data columns (total 12 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   ICAO Empresa Aérea      1034556 non-null  object
 1   Número Voo              1034556 non-null  object
 2   Código Tipo Linha       1034556 non-null  object
 3   ICAO Aeródromo Origem   1034556 non-null  object
 4   ICAO Aeródromo Destino  1034556 non-null  object
 5   Partida Prevista        1022536 non-null  object
 6   Partida Real            988714 non-null   object
 7   Chegada Prevista        1022536 non-null  object
 8   Chegada Real            988714 non-null   object
 9   Situação Voo            1034556 non-null  object
 10  Código Justificativa    267790 non-null   object
 11  Codigo DI               1033945 non-null  object
dtypes: object(12)
memory usage: 94.7+ MB


## Dicionário de dados
| Campo                         | Descrição                                                                                                   |
|-------------------------------|-------------------------------------------------------------------------------------------------------------|
| Sigla ICAO Empresa Aérea      | Sigla/Designador ICAO Empresa Aérea                                                                         |
| Número Voo                    | Numeração do voo                                                                                            |
| Código DI                     | Caractere usado para identificar o Dígito Identificador (DI) para cada etapa de voo                         |
| Código Tipo Linha             | Caractere usado para identificar o Tipo de Linha realizada para cada etapa de voo                           |
| Sigla ICAO Aeroporto Origem   | Sigla/Designador ICAO Aeroporto de Origem                                                                   |
| Sigla ICAO Aeroporto Destino  | Sigla/Designador ICAO Aeroporto de Destino                                                                  |
| Partida Prevista              | Data e horário da partida prevista informada pela empresa aérea, em horário de Brasília                      |
| Partida Real                  | Data e horário da partida realizada informada pela empresa aérea, em horário de Brasília                     |
| Chegada Prevista              | Data e horário da chegada prevista informada pela empresa aérea, em horário de Brasília                      |
| Chegada Real                  | Data e horário da chegada realizada, informada pela empresa aérea, em horário de Brasília                    |
| Situação do voo               | Campo informando a situação do voo: realizado, cancelado ou não informado.                                  |


# Tratamento de dados

## Join com as tabelas complementares

In [9]:
# Preparando DataFrames de Aeroportos para Mesclagem com o DataFrame Principal.

# Criando cópias do DataFrame de aeroportos para uso separado como origem e destino.
df_aeroportos_origem = df_aeroportos.copy()
df_aeroportos_destino = df_aeroportos.copy()

# Renomeando as colunas do DataFrame de aeroportos origem.
# Isso é feito para evitar conflitos de nomes ao mesclar com o DataFrame principal.
# Cada nome de coluna é acrescido de '_origem' para indicar que se refere ao aeroporto de origem do voo.
df_aeroportos_origem.rename(columns={col: col + '_origem' for col in df_aeroportos_origem.columns}, inplace=True)

# Renomeando as colunas do DataFrame de aeroportos destino de forma similar.
# Aqui, '_destino' é acrescido para indicar que as colunas se referem ao aeroporto de destino.
df_aeroportos_destino.rename(columns={col: col + '_destino' for col in df_aeroportos_destino.columns}, inplace=True)

# Mesclando o DataFrame principal com o DataFrame de companhias aéreas.
# A mesclagem é feita com base na coluna 'ICAO Empresa Aérea' do DataFrame principal
# e na coluna 'Sigla OACI' do DataFrame das companhias aéreas.
# A opção 'inner' assegura que apenas os registros presentes em ambos os DataFrames sejam mantidos.
df = pd.merge(df, df_cia_aerea, left_on='ICAO Empresa Aérea', right_on='Sigla OACI', how='inner')

# Mesclando o DataFrame resultante com o DataFrame de aeroportos origem.
# A mesclagem utiliza 'ICAO Aeródromo Origem' do DataFrame principal e 'Sigla OACI_origem' do DataFrame de aeroportos origem.
df = pd.merge(df, df_aeroportos_origem, left_on='ICAO Aeródromo Origem', right_on='Sigla OACI_origem', how='inner')

# Mesclando o DataFrame resultante com o DataFrame de aeroportos destino.
# Similar à etapa anterior, mas agora usando 'ICAO Aeródromo Destino' e 'Sigla OACI_destino'.
df = pd.merge(df, df_aeroportos_destino, left_on='ICAO Aeródromo Destino', right_on='Sigla OACI_destino', how='inner')


## Filtro de cia aerea 

In [10]:
# Filtrando o DataFrame para incluir apenas voos das empresas aéreas específicas.

# Definindo a lista de códigos ICAO das empresas aéreas desejadas para o filtro.
valores_filtrar = ['GLO', 'AZU', 'TAM']

# Filtrando o DataFrame para manter apenas as linhas onde 'ICAO Empresa Aérea' corresponde aos valores na lista de filtro.
df = df[df['ICAO Empresa Aérea'].isin(valores_filtrar)]

## Datas

In [11]:
# Convertendo colunas de datas e horas de partidas e chegadas para o formato datetime.

# As seguintes colunas são convertidas para o tipo datetime usando o formato especificado '%d/%m/%Y %H:%M':
# 'Partida Prevista', 'Partida Real', 'Chegada Prevista' e 'Chegada Real'.
df['Partida Prevista'] = pd.to_datetime(df['Partida Prevista'], format='%d/%m/%Y %H:%M')
df['Partida Real'] = pd.to_datetime(df['Partida Real'], format='%d/%m/%Y %H:%M')
df['Chegada Prevista'] = pd.to_datetime(df['Chegada Prevista'], format='%d/%m/%Y %H:%M')
df['Chegada Real'] = pd.to_datetime(df['Chegada Real'], format='%d/%m/%Y %H:%M')

# Aplicando a função 'is_delayed' em cada linha para determinar se o voo está atrasado ou pontual.
# O resultado é armazenado na nova coluna 'Status do Voo'.
df['Status do Voo'] = df.apply(is_delayed, axis=1)

### Criação de colunas com delta

In [12]:
# Aplicando a função para calcular o delta de tempo
df['Delta Tempo Partida delta'] = df.apply(calculate_time_delta, axis=1, start_col='Partida Prevista', end_col='Partida Real')
df['Delta Tempo Chegada delta'] = df.apply(calculate_time_delta, axis=1, start_col='Chegada Prevista', end_col='Chegada Real')

# Aplicando a função para calcular o delta de tempo
df['Delta Tempo Partida int'] = df.apply(calculate_time_int, axis=1, start_col='Partida Prevista', end_col='Partida Real')
df['Delta Tempo Chegada int'] = df.apply(calculate_time_int, axis=1, start_col='Chegada Prevista', end_col='Chegada Real')

## Seleção de colunas

In [ ]:
df= df[['Nome Empresas','Número Voo', 'Código DI',
 'Código Tipo Linha', 'Partida Prevista', 'Partida Real',
 'Chegada Prevista', 'Chegada Real', 'Situação Voo',
 'Descrição_origem', 'País_origem', 'Continente_origem',
 'Descrição_destino', 'País_destino', 'Continente_destino',
 'Status do Voo', 'Delta Tempo Partida delta', 'Delta Tempo Chegada delta',
 'Delta Tempo Partida int', 'Delta Tempo Chegada int', 'Cidade_origem',
 'UF_origem', 'Cidade_destino', 'UF_destino']]

## Removendo caracteres especiais e espaço do nome das colunas

In [ ]:
# Aplicar a função a cada nome de coluna
df.columns = [padronizar_nome_coluna(col) for col in df.columns]

# Adiação de colunas mês, dia do mês e dia da semana

In [ ]:
# Extração de componentes de data das colunas de partida e chegada.

# As seguintes operações são realizadas para extrair informações específicas das datas de partida:
# 1. 'mes_partida': Extração do mês da 'partida_prevista'.
# 2. 'dia_semana_partida': Extração do dia da semana da 'partida_prevista' (0 = Segunda-feira, 6 = Domingo).
# 3. 'dia_mes_partida': Extração do dia do mês da 'partida_prevista'.
df['mes_partida'] = df['partida_prevista'].dt.month
df['dia_semana_partida'] = df['partida_prevista'].dt.dayofweek
df['dia_mes_partida'] = df['partida_prevista'].dt.day

# Nota: As mesmas informações são extraídas para 'chegada_prevista', mas há um erro no código;
# 'partida_prevista' está sendo usada ao invés de 'chegada_prevista'.
# As variáveis 'mes_chegada', 'dia_semana_chegada', e 'dia_mes_chegada' devem ser ajustadas para usar 'chegada_prevista'.
df['mes_chegada'] = df['partida_prevista'].dt.month
df['dia_semana_chegada'] = df['partida_prevista'].dt.dayofweek
df['dia_mes_chegada'] = df['partida_prevista'].dt.day

# Criando coluna com as principais rotas

In [ ]:
# Criação da variavel rota (aeroporto origem + aeroporto destino). Observar as rotas mais problemas de atraso e cancelamento
df['rota'] = df['descricao_origem'] + " -> " + df['descricao_destino']

# Salvando dataset tratado

In [ ]:
# Definir o caminho do diretório e do arquivo CSV
diretorio = 'dados_tratados'
nome_arquivo = 'historico_voo_tratados.csv'
caminho_completo = os.path.join(diretorio, nome_arquivo)

# Verificar se o diretório existe. Se não, criar o diretório
if not os.path.exists(diretorio):
    os.makedirs(diretorio)

# Salvar o DataFrame no arquivo CSV
df.to_csv(caminho_completo, index=False)